## 2.6 Pipelines

### Problema

Tenemos nuestro modelo en producción. Nuestro API de Flask carga nuestro modelo, y tenemos un endpoint para enviar datos y realizar alguna predicción con éstos:

Lamentablemetne, nuestro API no es el más práctico del mundo. Para empezar, el formato en el que pide los datos es muy poco amigable. El JSON de entrada podría ser algo así:

```bash
curl -X POST http://127.0.0.1:5000/predecir -d '{"input": [0,1,0.6159084,0,0,0.55547282,1]}'
```

```json
{
  "Pclass": 0,
  "Sex": 1,
  "Age": 0.6159084,
  "SibSp": 0,
  "Parch": 0,
  "Fare": 0.55547282,
  "Embarked": 1
}
```

Esto estaría mucho mejor porque mínimo sabríamos los valores que estamos asignando a cada feature. No obstante,

Recordemos que en nuestro proceso de limpieza y feature engineering, realizamos transformaciones significativas a los datos: conversión a variables numéricas, normalización y escalamiento.

Por supuesto que no podemos esperar que los usuarios de nuestro API realicen estas transformaciones por su propia cuenta.

El objetivo final de esta implementación será que podamos usar nuestro API con un JSON de entrada como el siguiente:

Es decir, sin transformaciones.

Podríamos creer que implementaremos lógica en nuestro API para realizar transformaciones antes de hacer inferencia, pero la implementación que seguiremos es mucho más simple y elegante que eso.

```json
{
  "Pclass": 2,
  "Sex": "male",
  "Age": 46,
  "SibSp": 0,
  "Parch": 0,
  "Fare": 7.2500,
  "Embarked": "C"
}
```

¿Qué significa una edad de 0.6159084?

Lo que haremos a continuación es utilizar el objeto **Pipeline** de *Scikit Learn*, el cual, como su nombre lo sugiere, creará las directrices que permitirán el flujo de datos desde su entrada hasta su salida en forma de predicciones. La siguiente imagen lo ilustra muy bien:

*Fuente: Ravi Teja Kandimalla*

El código que escribiremos es parecido al que hemos escrito anteriormente, pero reescribiremos ciertas partes para que funcione correctamente el pipeline.

Crearemos nuestro Pipeline en un nuevo notebook. Abre el notebook `6_pipeline.ipynb`. Usaremos los datos que dejamos en el directorio `clean/`.

---

### Importar paquetes

```python
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder # <------------------ Esto es nuevo :)
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.preprocessing import QuantileTransformer
from sklearn.compose import ColumnTransformer # <------------------ Esto es nuevo :)
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline # <------------------ Esto es nuevo :)

# Modelos
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.naive_bayes import GaussianNB, BernoulliNB

# Métricas de evaluación
from sklearn.metrics import accuracy_score

# Para guardar el modelo
import pickle

df = pd.read_csv('./data/titanic_clean.csv')
```

---

### Preprocesamiento

Vemos aquí algo nuevo y algo “viejo”. En nuestros notebooks anteriores, usamos `QuantileTransformer` para normalizar las columnas Age y Fare. Posteriormente, creamos un nuevo DataFrame y lo guardamos con estas nuevas transformaciones.

El objeto **ColumnTransformer** define cuáles y cómo serán las transformaciones que se harán a los datos. Digamos que es nuestro proceso de *feature engineering* en un solo objeto.

En notebooks anteriores usamos `LabelEncoder`. No profundicemos en las diferencias técnicas. Para propósitos de nuestro proyecto `OneHotEncoder` y `LabelEncoder` hacen lo mismo. Tenemos que usar `OneHotEncoder` porque `LabelEncoder` no funciona con Pipelines.

```python
# Definir preprocesamiento
preprocessor = ColumnTransformer(
    transformers=[
        ('onehot', OneHotEncoder(), ['Sex', 'Embarked']),
        ('age', QuantileTransformer(output_distribution='normal', n_quantiles=500), ['Age']),
        ('fare', QuantileTransformer(output_distribution='normal', n_quantiles=500), ['Fare'])
    ],
    remainder='passthrough'  # Mantener otras columnas sin cambios
)
```

---

### Definición de modelos

Nuestro diccionario de modelos tiene un cambio pequeño:

```python
modelos = {
    'Regresión Logística': {
        'modelo': LogisticRegression(),
        'parametros': {
            'model__C': [0.01, 0.1, 1, 10, 100],
            'model__penalty': ['l1', 'l2'],
            'model__solver': ['liblinear', 'saga'],
            'model__max_iter': [100, 500, 1000]
        }
    },
    'Clasificador de Vectores de Soporte': {
        'modelo': SVC(),
        'parametros': {
            'model__kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
            'model__C': [0.1, 1, 10]
        }
    },
    'Clasificador de Árbol de Decisión': {
        'modelo': DecisionTreeClassifier(),
        'parametros': {
            'model__splitter': ['best', 'random'],
            'model__max_depth': [None, 1, 2, 3, 4]
        }
    },
    'Clasificador de Bosques Aleatorios': {
        'modelo': RandomForestClassifier(),
        'parametros': {
            'model__n_estimators': [10, 100],
            'model__max_depth': [None, 1, 2, 3, 4],
            'model__max_features': ['sqrt', 'log2', None]
        }
    },
    'Clasificador de Gradient Boosting': {
        'modelo': GradientBoostingClassifier(),
        'parametros': {
            'model__n_estimators': [10, 100],
            'model__max_depth': [None, 1, 2, 3, 4]
        }
    },
    'Clasificador AdaBoost': {
        'modelo': AdaBoostClassifier(),
        'parametros': {
            'model__n_estimators': [10, 100]
        }
    },
    'Clasificador K-Nearest Neighbors': {
        'modelo': KNeighborsClassifier(),
        'parametros': {
            'model__n_neighbors': [3, 5, 7]
        }
    },
    'Clasificador XGBoost': {
        'modelo': XGBClassifier(),
        'parametros': {
            'model__n_estimators': [10, 100],
            'model__max_depth': [None, 1, 2, 3]
        }
    },
    'Clasificador LGBM': {
        'modelo': LGBMClassifier(),
        'parametros': {
            'model__n_estimators': [10, 100],
            'model__max_depth': [None, 1, 2, 3],
            'model__learning_rate': [0.1, 0.2, 0.3],
            'model__verbose': [-1]
        }
    },
    'GaussianNB': {
        'modelo': GaussianNB(),
        'parametros': {}
    },
    'Clasificador Naive Bayes': {
        'modelo': BernoulliNB(),
        'parametros': {
            'model__alpha': [0.1, 1.0, 10.0]
        }
    }
}
```

---

### División de datos

Ningún cambio significativo en esta etapa.

```python
X = df.drop(['Survived'], axis=1)
y = df['Survived']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=100)
```

### Variables auxiliares

Ningún cambio significativo en esta etapa.

---

### Ciclo for de GridSearch

El contenido de nuestro ciclo for sí cambiará un poco:

```python
puntajes_modelos = []
mejor_precision = 0
mejor_estimador = None
mejor_modelo = None
estimadores = {}

for nombre, info_modelo in modelos.items():
    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('scaler', MinMaxScaler()),  # MinMaxScaler se aplica a todas las columnas después del pre
        ('model', info_modelo['modelo'])
    ])
    
    grid_search = GridSearchCV(
        estimator=pipeline,
        param_grid=info_modelo['parametros'],
        cv=5,
        scoring='accuracy',
        verbose=0,
        n_jobs=-1,
    )

    grid_search.fit(X_train, y_train)
    y_pred = grid_search.predict(X_test)
    precision = accuracy_score(y_test, y_pred)

    puntajes_modelos.append({
        'Modelo': nombre,
        'Precisión': precision
    })

    estimadores[nombre] = grid_search.best_estimator_

    if precision > mejor_precision:
        mejor_modelo = nombre
        mejor_precision = precision
        mejor_estimador = grid_search.best_estimator_
```

Los pasos (`steps`) que especificamos en este pipeline son:

1. Preprocesar los datos haciendo uso del objeto `preprocessor` que creamos anteriormente.
2. Escalamiento de datos usando `MinMaxScaler`.
3. Modelo.

Posteriormente, usaremos este nuevo objeto llamado `pipeline` como si fuera el modelo o estimador.

Vemos que esto es prácticamente igual a lo que hacíamos antes. La diferencia está en que `estimator` ya no es un modelo individual sino un objeto de tipo `Pipeline`.

---

### Mostrar resultados y guardar modelo

Lo último no sufre ningún cambio:

```python
metricas = pd.DataFrame(puntajes_modelos).sort_values('Precisión', ascending=False)

print("Rendimiento de los modelos de clasificación")
print(metricas.round(2))

print('---------------------------------------------------')
print("MEJOR MODELO DE CLASIFICACIÓN")
print(f"Modelo: {mejor_modelo}")
print(f"Precisión: {mejor_precision:.2f}")

with open('pipeline.pkl', 'wb') as archivo_estimador:
    pickle.dump(mejor_estimador, archivo_estimador)
```

#### Warnings

Es muy probable que nuestro GridSearch arroje varios *warnings*:

> The max\_iter was reached which means the coef\_ did not converge
> The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME alg

Es importante leer los warnings, pero no hay que preocuparnos tanto por ellos. Indican que en alguno de los hiperparámetros probados no fueron adecuados.

Aunque el código es el mismo, cuando guardamos nuestro pickle, estamos guardando un **pipeline**, no un modelo individual.

---

### Refactorizar API

Finalmente, modificaremos el código de nuestro API para que funcione con este pipeline. Recordemos que el objetivo será usar JSONs de entrada como éste:

```json
{
  "Pclass": 2,
  "Sex": "male",
  "Age": 46,
  "SibSp": 0,
  "Parch": 0,
  "Fare": 7.2500,
  "Embarked": "C"
}
```

Nuestro archivo `app.py` ahora es el siguiente:

```python
from flask import Flask, request, jsonify
import pickle
import pandas as pd

app = Flask(__name__)

# Cargar el modelo guardado
with open('pipeline.pkl', 'rb') as archivo_modelo:
    modelo = pickle.load(archivo_modelo)

@app.route('/predecir', methods=['POST'])
def predecir():
    # Obtener los datos de la solicitud
    data = request.get_json()

    # Crear un DataFrame de pandas a partir del JSON
    input_data = pd.DataFrame([data])

    # Hacer la predicción usando el modelo que tiene el pipeline que hará la transformación
    prediccion = modelo.predict(input_data)

    # Devolver la predicción como JSON
    output = {'Survived': int(prediccion[0])}
    return jsonify(output)

if __name__ == '__main__':
    app.run(debug=True)
```

Si probamos nuestra app de Flask ahora con el JSON de entrada deseado, veremos que funciona perfectamente:

```bash
curl -X POST http://127.0.0.1:5000/predecir -H 'Content-Type: application/json' -d '{
  "Pclass": 2,
  "Sex": "male",
  "Age": 46,
  "SibSp": 0,
  "Parch": 0,
  "Fare": 7.2500,
  "Embarked": "C"
}'
```

¡Nuestro Pipeline funciona perfectamente!


In [1]:
# =========================
# 2.6 Pipelines - Entrenamiento
# =========================

# ---- Importar paquetes
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np

# Preprocesamiento
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler, QuantileTransformer
from sklearn.compose import ColumnTransformer

# Model selection
from sklearn.model_selection import train_test_split, GridSearchCV

# Pipeline
from sklearn.pipeline import Pipeline

# Modelos base de sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB, BernoulliNB

# Métrica
from sklearn.metrics import accuracy_score

# Guardado
import pickle

# ---- (Opcional) Modelos adicionales: XGBoost y LightGBM
have_xgb = False
have_lgbm = False
try:
    from xgboost import XGBClassifier
    have_xgb = True
except Exception:
    pass

try:
    from lightgbm import LGBMClassifier
    have_lgbm = True
except Exception:
    pass

# -----------------------
# Carga de datos
# -----------------------
# Asegúrate de que la ruta exista y el CSV contenga las columnas esperadas:
# ['Pclass','Sex','Age','SibSp','Parch','Fare','Embarked','Survived']
df = pd.read_csv('./data/titanic_clean.csv')

# -----------------------
# Preprocesamiento
# -----------------------
# - OneHotEncoder para columnas categóricas
# - QuantileTransformer para 'Age' y 'Fare'
# - remainder='passthrough' mantiene el resto de columnas
preprocessor = ColumnTransformer(
    transformers=[
        ('onehot', OneHotEncoder(handle_unknown='ignore'), ['Sex', 'Embarked']),
        ('age', QuantileTransformer(output_distribution='normal', n_quantiles=500, subsample=int(1e9)), ['Age']),
        ('fare', QuantileTransformer(output_distribution='normal', n_quantiles=500, subsample=int(1e9)), ['Fare']),
    ],
    remainder='passthrough'
)

# -----------------------
# Definición de modelos e hiperparámetros
# Los hiperparámetros se referencian con el prefijo 'model__'
# porque el estimador final está dentro del Pipeline en el paso 'model'.
# -----------------------
modelos = {
    'Regresión Logística': {
        'modelo': LogisticRegression(),
        'parametros': {
            'model__C': [0.01, 0.1, 1, 10, 100],
            'model__penalty': ['l1', 'l2'],
            'model__solver': ['liblinear', 'saga'],
            'model__max_iter': [100, 500, 1000]
        }
    },
    'Clasificador de Vectores de Soporte': {
        'modelo': SVC(),
        'parametros': {
            'model__kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
            'model__C': [0.1, 1, 10]
        }
    },
    'Clasificador de Árbol de Decisión': {
        'modelo': DecisionTreeClassifier(),
        'parametros': {
            'model__splitter': ['best', 'random'],
            'model__max_depth': [None, 1, 2, 3, 4]
        }
    },
    'Clasificador de Bosques Aleatorios': {
        'modelo': RandomForestClassifier(),
        'parametros': {
            'model__n_estimators': [50, 100, 200],
            'model__max_depth': [None, 3, 5, 7],
            'model__max_features': ['sqrt', 'log2', None]
        }
    },
    'Clasificador de Gradient Boosting': {
        'modelo': GradientBoostingClassifier(),
        'parametros': {
            'model__n_estimators': [50, 100, 200],
            'model__max_depth': [1, 2, 3, 4]
        }
    },
    'Clasificador AdaBoost': {
        'modelo': AdaBoostClassifier(),
        'parametros': {
            'model__n_estimators': [50, 100, 200]
        }
    },
    'Clasificador K-Nearest Neighbors': {
        'modelo': KNeighborsClassifier(),
        'parametros': {
            'model__n_neighbors': [3, 5, 7, 9]
        }
    },
    'GaussianNB': {
        'modelo': GaussianNB(),
        'parametros': {}  # sin hiperparámetros en este ejemplo
    },
    'Clasificador Naive Bayes (Bernoulli)': {
        'modelo': BernoulliNB(),
        'parametros': {
            'model__alpha': [0.1, 0.5, 1.0, 10.0]
        }
    },
}

# Incluir XGBoost si está disponible
if have_xgb:
    modelos['Clasificador XGBoost'] = {
        'modelo': XGBClassifier(
            eval_metric='logloss',
            use_label_encoder=False,
            n_estimators=100,
            n_jobs=-1
        ),
        'parametros': {
            'model__n_estimators': [100, 200],
            'model__max_depth': [2, 3, 4, 5]
        }
    }

# Incluir LightGBM si está disponible
if have_lgbm:
    modelos['Clasificador LGBM'] = {
        'modelo': LGBMClassifier(),
        'parametros': {
            'model__n_estimators': [100, 200],
            # En LightGBM, max_depth=-1 es "sin límite"
            'model__max_depth': [-1, 2, 3, 4],
            'model__learning_rate': [0.05, 0.1, 0.2],
            'model__verbose': [-1]
        }
    }

# -----------------------
# División de datos
# -----------------------
X = df.drop(['Survived'], axis=1)
y = df['Survived']
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=100, stratify=y
)

# -----------------------
# Entrenamiento con GridSearch en un bucle por modelo
# Cada iteración crea un Pipeline: preprocessor -> MinMaxScaler -> model
# Guardamos el mejor Pipeline encontrado (mejor_estimador).
# -----------------------
puntajes_modelos = []
mejor_precision = 0.0
mejor_estimador = None
mejor_modelo = None
estimadores = {}

for nombre, info in modelos.items():
    print(f"\n=== Entrenando: {nombre} ===")

    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('scaler', MinMaxScaler()),
        ('model', info['modelo'])
    ])

    grid_search = GridSearchCV(
        estimator=pipeline,
        param_grid=info['parametros'],
        cv=5,
        scoring='accuracy',
        verbose=0,
        n_jobs=-1,
    )

    grid_search.fit(X_train, y_train)
    y_pred = grid_search.predict(X_test)
    precision = accuracy_score(y_test, y_pred)

    print(f"Mejores params: {grid_search.best_params_}")
    print(f"Accuracy (test): {precision:.4f}")

    puntajes_modelos.append({'Modelo': nombre, 'Precisión': precision})
    estimadores[nombre] = grid_search.best_estimator_

    if precision > mejor_precision:
        mejor_precision = precision
        mejor_modelo = nombre
        mejor_estimador = grid_search.best_estimator_

# -----------------------
# Resultados
# -----------------------
metricas = pd.DataFrame(puntajes_modelos).sort_values('Precisión', ascending=False)
print("\nRendimiento de los modelos de clasificación")
print(metricas.round(4))
print('---------------------------------------------------')
print("MEJOR MODELO DE CLASIFICACIÓN")
print(f"Modelo: {mejor_modelo}")
print(f"Precisión: {mejor_precision:.4f}")

# -----------------------
# Guardar el mejor Pipeline
# -----------------------
with open('pipeline.pkl', 'wb') as f:
    pickle.dump(mejor_estimador, f)

print("\n✅ Guardado: pipeline.pkl (contiene TODO el Pipeline: preprocesamiento + modelo).")



=== Entrenando: Regresión Logística ===
Mejores params: {'model__C': 0.01, 'model__max_iter': 100, 'model__penalty': 'l2', 'model__solver': 'liblinear'}
Accuracy (test): 0.8324

=== Entrenando: Clasificador de Vectores de Soporte ===
Mejores params: {'model__C': 1, 'model__kernel': 'poly'}
Accuracy (test): 0.8380

=== Entrenando: Clasificador de Árbol de Decisión ===
Mejores params: {'model__max_depth': 4, 'model__splitter': 'random'}
Accuracy (test): 0.8436

=== Entrenando: Clasificador de Bosques Aleatorios ===
Mejores params: {'model__max_depth': 7, 'model__max_features': 'sqrt', 'model__n_estimators': 50}
Accuracy (test): 0.8827

=== Entrenando: Clasificador de Gradient Boosting ===
Mejores params: {'model__max_depth': 3, 'model__n_estimators': 100}
Accuracy (test): 0.8492

=== Entrenando: Clasificador AdaBoost ===
Mejores params: {'model__n_estimators': 200}
Accuracy (test): 0.8212

=== Entrenando: Clasificador K-Nearest Neighbors ===
Mejores params: {'model__n_neighbors': 9}
Acc

**Para la prueba:**

```shell
Invoke-RestMethod -Uri http://127.0.0.1:5000/predecir `
    -Method Post `
    -ContentType "application/json" `
    -Body '{
        "Pclass": 2,
        "Sex": "male",
        "Age": 46,
        "SibSp": 0,
        "Parch": 0,
        "Fare": 7.2500,
        "Embarked": "C"
    }'
```